In [38]:
import pandas as pd
import numpy as np
import string as s
import seaborn as sns
from shutil import copyfile
import math
# import tables
import matplotlib
import matplotlib.pyplot as plt
from itertools import groupby
import scipy.stats as stats
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'


targeted sequencing QC

In [2]:
wkdir = '/home/szong/projects/QC/targeted_seq/BLGSP/'

In [4]:
# f1 = '{0}{1}'.format(wkdir, 'NCI_capture_of_BLGSP_samples_level_2_QC_results_9_runs.xls')
f1 = '{0}{1}'.format(wkdir, 'NCI_capture_of_BLGSP_samples_level_2_QC_results_11_runs_2018_04_09.xls')
df1 = pd.read_excel(f1, sheetname='result').dropna()
df1.head(2)

,pool library,library,sample_id,index,# of on-target reads,# of off-target reads,% of reads on target (from total reads),% of reads off target (from total reads),fold coverage (incl. duplicate reads)
0,IX6039.CC47LANXX_1.1stLane,A78672,BLGSP-71-19-00106,CCAGCA,5358375.0,3674778.0,0.593190,0.406810,138.0
1,IX6039.CC47LANXX_1.1stLane,A78675,BLGSP-71-19-00114,TACAAG,8695613.0,5051912.0,0.632522,0.367478,225.0


In [5]:
df1.columns

Index(['pool library', 'library', 'sample_id', 'index', '# of on-target reads',
       '# of off-target reads', '% of reads on target (from total reads)',
       '% of reads off target (from total reads)',
       'fold coverage (incl. duplicate reads)'],
      dtype='object')

In [6]:
df_cov = df1.groupby('library').agg({'fold coverage (incl. duplicate reads)':sum,\
                            '% of reads on target (from total reads)':'mean',\
                            '% of reads off target (from total reads)': 'mean'})

In [7]:
df_cov.head(2)

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads)
library,,,
A78670,724.0,0.617571,0.382429
A78671,490.0,0.620799,0.379201


In [8]:
df_cov.shape

(76, 3)

In [9]:
df_cov.index

Index(['A78670', 'A78671', 'A78672', 'A78673', 'A78674', 'A78675', 'A78676',
       'A78677', 'A78678', 'A78679', 'A78682', 'A78683', 'A78686', 'A78691',
       'A82260', 'A82261', 'A82262', 'A82263', 'A82264', 'A82265', 'A82266',
       'A82267', 'A82268', 'A82269', 'A82270', 'A82271', 'A82272', 'A82273',
       'A82274', 'A82275', 'A82276', 'A82277', 'A82278', 'A82279', 'A82280',
       'A82281', 'A82282', 'A82283', 'A82284', 'A82285', 'A82286', 'A82287',
       'A82288', 'A82289', 'A85121', 'A85122', 'A85123', 'A85124', 'A85125',
       'A85126', 'A85127', 'A85128', 'A85129', 'A85130', 'A85131', 'A85132',
       'A85133', 'A85134', 'A85135', 'A85184', 'A85185', 'A87321', 'A87322',
       'A87323', 'A87324', 'A87325', 'A87326', 'A87327', 'A87328', 'A87329',
       'A87330', 'A88446', 'A88447', 'A88448', 'A88449', 'A88450'],
      dtype='object', name='library')

In [10]:
# f2 = '{0}{1}'.format(wkdir, 'SeqVal121.RawAlignedBaseCount.20180306.xlsx')
f2 = '{0}{1}'.format(wkdir, 'SeqVal121.RawAlignedBaseCount-IX6191.20180507.xlsx')
df2 = pd.read_excel(f2, sheetname='raw.out.IX6191').dropna()
df2.head(2)

,Pool,Library,Run ID,Raw Reads,Aligned Reads,Raw Bases Sequenced,Number of bases Aligned
0,IX6191,A88446,run224097,43140955,43091101,5392619375,5386387625
1,IX6191,A88450,run224097,50147691,49891003,6268461375,6236375375


In [11]:
df2.columns

Index(['Pool', 'Library', 'Run ID', 'Raw Reads', 'Aligned Reads',
       'Raw Bases Sequenced', 'Number of bases Aligned'],
      dtype='object')

In [12]:
df3 = df2.groupby('Library').agg({'Raw Bases Sequenced':sum, 'Number of bases Aligned':sum})
df3.head(2)

,Raw Bases Sequenced,Number of bases Aligned
Library,,
A87321,10649838250,10636656625
A87322,9193753875,9122026625


In [13]:
df_bases = df3/1e9

In [14]:
dfm1 = pd.merge(df_cov, df_bases, left_index=True, right_index=True)
dfm1.head(2)

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads),Raw Bases Sequenced,Number of bases Aligned
A87321,590.0,0.505099,0.494901,10.649838,10.636657
A87322,549.0,0.550941,0.449059,9.193754,9.122027


In [27]:
dfm1.to_csv()

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads),Raw Bases Sequenced,Number of bases Aligned
A87321,590.0,0.505099,0.494901,10.649838,10.636657
A87322,549.0,0.550941,0.449059,9.193754,9.122027
A87323,603.0,0.512159,0.487841,10.748379,10.735394
A87324,476.0,0.560508,0.439492,7.848406,7.796502
A87325,549.0,0.512200,0.487800,9.753617,9.741567
A87326,551.0,0.588321,0.411679,8.625918,8.617134
A87327,748.0,0.494742,0.505258,13.786407,13.769868
A87328,617.0,0.573535,0.426465,9.807973,9.796177
A87329,644.0,0.583527,0.416473,10.081224,10.069572
A87330,567.0,0.565608,0.434391,9.128201,9.117313


where to get the *dOutput_parsed.tsv file?, this is the file to find the bases with 0 coverage, Kane's team make these files

In [99]:
# f4 = '{0}{1}'.format(wkdir, 'probe_cov_files.txt')
f4 = '{0}{1}'.format(wkdir,'run1_files.txt')
df4 =  pd.read_csv(f4, header=None)
df4.columns = ['file']
df4.head(2)

,file
0,A87321_1.paired.dup.sorted_parsed.tsv
1,A87322_1.paired.dup.sorted_parsed.tsv


In [100]:
def get_ocov_bases(df4):
    lst = []
    for idx, f in df4.iterrows():
        file = f['file']
        lib = file.split('.')[0]
        df = pd.read_csv('{0}{1}'.format(wkdir, file), sep='\t')
        bases_w_ocov = df.loc[0,'Number of Bases With This Coverage (Or More) (Not Genomically Unique)']
        lst.append([lib, bases_w_ocov])
    dff = pd.DataFrame(lst)
    dff.columns = ['library', 'bases_w_ocoverage']
    dff = dff.set_index('library', drop=True)
    return dff

In [101]:
dff = get_ocov_bases(df4).reset_index()

In [102]:
dff.head(2)

,library,bases_w_ocoverage
0,A87321_1,17507
1,A87322_1,44099


In [103]:
dff['lib'] = dff.reset_index()['library'].str.replace('_1', '')
dff.drop('library', axis=1, inplace=True)

In [104]:
dff

,bases_w_ocoverage,lib
0,17507,A87321
1,44099,A87322
2,18024,A87323
3,43850,A87324
4,17095,A87325
5,49697,A87326
6,16523,A87327
7,18328,A87328
8,15724,A87329
9,18515,A87330


In [98]:
dff2 = dff.copy()

In [106]:
dff3 = pd.merge(dff, dff2, left_on='lib', right_on='lib')

In [110]:
dff3['0cov'] = dff3.min(axis=1)

In [114]:
dff3.to_csv('{}zero_coverage_base_batch2.txt'.format(wkdir), sep='\t')

In [24]:
dfm1

,fold coverage (incl. duplicate reads),% of reads on target (from total reads),% of reads off target (from total reads),Raw Bases Sequenced,Number of bases Aligned
A87321,590.0,0.505099,0.494901,10.649838,10.636657
A87322,549.0,0.550941,0.449059,9.193754,9.122027
A87323,603.0,0.512159,0.487841,10.748379,10.735394
A87324,476.0,0.560508,0.439492,7.848406,7.796502
A87325,549.0,0.512200,0.487800,9.753617,9.741567
A87326,551.0,0.588321,0.411679,8.625918,8.617134
A87327,748.0,0.494742,0.505258,13.786407,13.769868
A87328,617.0,0.573535,0.426465,9.807973,9.796177
A87329,644.0,0.583527,0.416473,10.081224,10.069572
A87330,567.0,0.565608,0.434391,9.128201,9.117313


In [26]:
dff.head(2)

,bases_w_ocoverage
library,
A78670,13358
A78671,18635


In [19]:
dfm2 = pd.merge(dfm1, dff, left_index=True, right_index=True)

In [20]:
dfm2 = dfm2.sort_index()

In [28]:
of = '{0}{1}'.format(wkdir, 'qc.txt')
dfm1.to_csv(of, sep='\t')

In [50]:
"{0:.4f}".format(6, 9)

'6.0000'

In [44]:
arr = np.array([4, 5,6,7, 44, 22, 11, 16])

In [41]:
((arr >= 15) & (arr <= 17)).sum()

0

In [45]:
(arr >= 15) & (arr <= 17)

array([False, False, False, False, False, False, False,  True], dtype=bool)

# merge targeted sequencing stats, zerocoverage bases

In [22]:
wkdir = '/home/szong/projects/QC/targeted_seq/BLGSP/'
f = '{}IX6191.indexMappingInternalOnly-run224097.txt'.format(wkdir)
f

'/home/szong/projects/QC/targeted_seq/BLGSP/IX6191.indexMappingInternalOnly-run224097.txt'

In [23]:
df = pd.read_csv(f, sep='\t')[['Sub-Library', 'Index']]
df.head(2)

,Sub-Library,Index
0,A87323,AAACCT
1,A88450,AAGCTA


In [45]:
run1_files = '{}run1_stats_files.txt'.format(wkdir)
run2_files = '{}run2_stats_files.txt'.format(wkdir)
# qc_path = '/home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run1/'
qc_path = '/home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run2/'

In [46]:
lst = []
with open(run2_files) as fp:  
    for cnt, line in enumerate(fp):
        idx = line.split('_')[2]
        f = '{0}{1}'.format(qc_path, line)
        lst.append([idx, f])
df1 = pd.DataFrame(lst)
df1.head(2)
    


,0,1
0,AAACCT,/home/sequence/Sequencing_Validation/SVA02/Seq...
1,AAGCTA,/home/sequence/Sequencing_Validation/SVA02/Seq...


In [47]:
df3 = pd.merge(df, df1, left_on='Index', right_on=0)
df3.head(2)

,Sub-Library,Index,0,1
0,A87323,AAACCT,AAACCT,/home/sequence/Sequencing_Validation/SVA02/Seq...
1,A88450,AAGCTA,AAGCTA,/home/sequence/Sequencing_Validation/SVA02/Seq...


In [48]:
for ix, rc in df3.iterrows():
    lib = rc['Sub-Library']
    f = rc[1].strip('\n')
    map_f = '{0}{1}_2.paired.dup.sorted_parsed.tsv'.format(wkdir, lib)

    print(lib, f, map_f)
    copyfile(f, map_f)

A87323 /home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run2/s_3_AAACCT_paired.dup.sorted_parsed.tsv /home/szong/projects/QC/targeted_seq/BLGSP/A87323_2.paired.dup.sorted_parsed.tsv
A88450 /home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run2/s_3_AAGCTA_paired.dup.sorted.dOutput_parsed.tsv /home/szong/projects/QC/targeted_seq/BLGSP/A88450_2.paired.dup.sorted_parsed.tsv
A87329 /home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run2/s_3_ATGGCA_paired.dup.sorted_parsed.tsv /home/szong/projects/QC/targeted_seq/BLGSP/A87329_2.paired.dup.sorted_parsed.tsv
A88446 /home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformatic Analysis/bedtoolsStats/IX6191-run2/s_3_CCTATT_paired.dup.sorted_parsed.tsv /home/szong/projects/QC/targeted_seq/BLGSP/A88446_2.paired.dup.sorted_parsed.tsv
A87325 /home/sequence/Sequencing_Validation/SVA02/SeqVal121/Bioinformati